<a href="https://colab.research.google.com/github/littleactuary/EAA/blob/main/ESG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from scipy.interpolate import interp1d, CubicSpline


### Chọn stock

In [2]:
stock_symbol = 'AAPL' #apple
data = yf.download(stock_symbol, start='2020-01-01', end='2023-12-31', interval='1mo')
data = data['Adj Close']

[*********************100%%**********************]  1 of 1 completed


## Hàm tạo giá của 1000 kịch bản

In [5]:

def generate_price_scenarios(data, n_scenarios, n_months,formatted_dates,scenario_names):
  # Tính tỷ suất sinh lợi hàng tháng và độ biến động (volatility của price)
  returns = data.pct_change().dropna()
  volatility = returns.std()
  # Tính toán drift
  drift = returns.mean() - 0.5 * volatility ** 2
  # Make scenario df
  scenarios = np.zeros((n_scenarios, n_months))
  scenarios[:, 0] = initial_price

  for t in range(1, n_months):
      z = np.random.standard_normal(n_scenarios)
      scenarios[:, t] = scenarios[:, t - 1] * np.exp(drift + volatility * z)

  scenario_prices_df = pd.DataFrame(scenarios, columns=formatted_dates, index=scenario_names)
  return scenario_prices_df




## Hàm tạo 1 yield curve chung cho 1000 kịch bản

In [6]:
def generate_discount_rate_scenarios1(time,yield_percent, n_months,formatted_dates,scenario_names):
  # Thông tin về thời gian
  # Chuyển đổi yields thành discount factors
  discount_factors = np.exp(-(yield_percent / 100) * time)

  # Interpolation
  # interp_func = interp1d(time, discount_factors, kind='linear', fill_value="extrapolate")
  interp_func = interp1d(time, discount_factors, kind='cubic', fill_value="extrapolate")
  times_in_years = [(date - dates[0]).days / 365.25 for date in dates]

  # Interpolate các discount factors cho mỗi tháng
  interpolated_discount_factors = interp_func(times_in_years)
  interpolated_discount_factors[0] = 1.0  # Đặt cột đầu tiên là 1

  # Tạp DataFrame
  discount_factors_df1 = pd.DataFrame(np.tile(interpolated_discount_factors, (1000, 1)),index=scenario_names, columns=formatted_dates)
  return discount_factors_df1



## Hàm tạo 1000 đường yield curve cho 1000 kịch bản

In [10]:
def vasicek_model(r0, alpha, b, sigma, n_scenarios, T=1, dt=1/12 ):
    t = np.linspace(0, T, int(T/dt)+1)  # Tạo các mốc thời gian từ 0 đến T với bước dt
    rates = np.zeros((n_scenarios, t.shape[0]))  # Tạo mảng để lưu trữ các lãi suất mô phỏng
    rates[:, 0] = r0  # Gán lãi suất ban đầu cho tất cả các kịch bản
    for i in range(1, t.shape[0]):
        dt = t[i] - t[i-1]  # Tính bước thời gian
        dW = np.random.standard_normal(size=n_scenarios) * np.sqrt(dt)  # Tạo nhiễu ngẫu nhiên
        rates[:, i] = rates[:, i-1] + alpha * (b - rates[:, i-1]) * dt + sigma * dW  # Tính lãi suất tại thời điểm tiếp theo
    return rates, t


def generate_discount_rate_scenarios2(r0, alpha, b, sigma,dates,n_scenarios,formatted_dates,scenario_names, T=1, dt=1/12):
  # simulate yield_curves
  simulated_yield_curves, t = vasicek_model(r0, alpha, b, sigma,n_scenarios)

  # yield curve to discount factor
  discount_factors = np.exp(-simulated_yield_curves * np.reshape(t, (1, -1)))

  # Thêm tháng đầu tiền là 1%
  discount_factors[:, 0] = 1.0

  # Dataframe
  discount_factors_df2 = pd.DataFrame(discount_factors,index=scenario_names, columns=formatted_dates)
  return discount_factors_df2



# Main

In [17]:


np.random.seed(42)

# Số lượng scenario và số tháng dự báo
n_scenarios = 1000
n_months = 13
initial_price = 100

# Cài đặt thời gian
dates = pd.date_range(start='2024-06-30', periods=13, freq='M')
formatted_dates = [date.strftime('%d-%m-%y') for date in dates]

# Scnario name
scenario_names = [f'Scenario {i+1}' for i in range(n_scenarios)]


# Yield curve ( Có thể thành input)
time = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yield_percent = np.array([5.366, 5.359, 5.363, 5.383, 5.332, 5.124, 4.749, 4.546, 4.369, 4.369, 4.392, 4.655, 4.554])


# Các par cho model Vasicek
alpha = 0.1
b = 0.03
sigma = 0.02
r0 = 0.03  # Lãi suất ban đầu

def generate_scenario():
  scenarios_price_df = generate_price_scenarios(data, n_scenarios, n_months,formatted_dates,scenario_names)
  discount_factors_df1 = generate_discount_rate_scenarios1(time,yield_percent, n_months,formatted_dates,scenario_names)
  discount_factors_df2 = generate_discount_rate_scenarios2(r0, alpha, b, sigma,dates,n_scenarios,formatted_dates,scenario_names, T=1, dt=1/12)

  with pd.ExcelWriter('economic_scenarios.xlsx') as writer:
    scenarios_price_df.to_excel(writer, sheet_name='Indice act_hors div',startrow=5)
    discount_factors_df1.to_excel(writer, sheet_name='Déflateurs1',startrow=5)
    discount_factors_df2.to_excel(writer, sheet_name='Déflateurs2',startrow=5)


  return 'economic_scenarios.xlsx'


In [18]:
generate_scenario()

'economic_scenarios.xlsx'

# Test

In [19]:
import pandas as pd
class actions():
    def __init__(self,address='test.xlsx') -> None:  # Khai báo
        df1=pd.read_excel(address,sheet_name='Indice act_hors div',skiprows=5,index_col='Unnamed: 0')
        # df2=pd.read_excel(address,sheet_name='Tx_div',skiprows=5,index_col='Unnamed: 0')
        df3=pd.read_excel(address,sheet_name='Déflateurs1',skiprows=5,index_col='Unnamed: 0')



        df3.columns=df1.columns

        indice_capitalisant = pd.DataFrame(columns=df1.columns,index=df1.index)
        indice_capitalisant[indice_capitalisant.columns[0]]=df1[df1.columns[0]]
        for i in range(1,df1.shape[1]):
            indice_capitalisant[df1.columns[i]] = indice_capitalisant[df1.columns[i-1]]*((df1[df1.columns[i]]/df1[df1.columns[i-1]])+((1)**(1/12)-1))

        #need
        self.indice_capitalisant_deflate=indice_capitalisant[indice_capitalisant.columns]* df3
        self.indice_capitalisant = indice_capitalisant
        self.martingale()



    def martingale(self,eurreur=0.02,valid_number=10**(-5)):
        # self.res=pd.DataFrame(self.indice_capitalisant_deflate.mean(),columns=['moyenne(VM actualisée)'])
        result= pd.DataFrame([self.indice_capitalisant_deflate.mean().values],index=['moyenne(VM actualisée)'])
        result.loc['VM initiale']=100
        result.loc['borne sup +2%']=result.loc['VM initiale']*(1+eurreur)
        result.loc['borne inf -2%']=result.loc['VM initiale']*(1-eurreur)
        result.loc['ecart relatif']=abs((result.loc['moyenne(VM actualisée)']-result.loc['VM initiale'])/result.loc['VM initiale'])



        self.ecart_relatif_moyen=float(result.loc['ecart relatif'].mean())
        self.ecart_relatif_moyen2=float(((result.loc['ecart relatif']**2).sum()/len(result.loc['ecart relatif']))**0.5)
        result.loc['taux']=result.loc['moyenne(VM actualisée)']/result.loc['VM initiale']

        #check test
        self.valid=True if self.ecart_relatif_moyen<valid_number else False
        self.result=result




In [20]:
a = actions(generate_scenario())

In [21]:
a.result

,0,1,2,3,4,5,6,7,8,9,10,11,12
moyenne(VM actualisée),100.0,101.715671,103.878261,106.350710,108.258599,109.584335,111.732009,114.212789,116.294790,118.110087,120.892420,122.803770,124.793632
VM initiale,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
borne sup +2%,102.0,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000
borne inf -2%,98.0,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
ecart relatif,0.0,0.017157,0.038783,0.063507,0.082586,0.095843,0.117320,0.142128,0.162948,0.181101,0.208924,0.228038,0.247936
taux,1.0,1.017157,1.038783,1.063507,1.082586,1.095843,1.117320,1.142128,1.162948,1.181101,1.208924,1.228038,1.247936
